In [13]:
import pandas as pd 
from sklearn.model_selection import train_test_split,KFold
from sklearn.linear_model import LinearRegression
import sklearn.metrics as mt

In [4]:
data = pd.read_csv(r"C:\Users\Kemalettin\Desktop\anlasekon\Python ile Makine Öğrenmesi\Advertising.csv",index_col=0)
data2 = data.copy() 
# artık genel adım gibi oldu bunları bundan sonra açıklamayacağım gereksiz uzun olmasın
data2.head()

,TV,Radio,Newspaper,Sales
1,230.1,37.8,69.2,22.1
2,44.5,39.3,45.1,10.4
3,17.2,45.9,69.3,9.3
4,151.5,41.3,58.5,18.5
5,180.8,10.8,58.4,12.9


In [16]:
# değişkenleri tanımlama
y = data2["Sales"]
X = data2.drop(columns="Sales",axis = 1)

# Veriyi bölme
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=3)

# modeli oluşturma
lr = LinearRegression().fit(X_train,y_train)


In [10]:
# işleri artık düzgünce çözelim tüm araçları kullanabilelim
def score(model,x_train,x_test,y_train,y_test):
    """ 
    Parameters:

    x_train: Training için bölünmüş X veri kümesi
    x_test: Test için bölünmüş X veri kümesi
    y_train: Training için bölünmüş y veri kümesi
    y_test: Test için bölünmüş y veri kümesi

    Returns:

    r2_training_score, r2_test_score, mse_training_score, mse_test_score
    r2_training_score: Training verisinde elde edilen R2 puanı
    r2_test_score: Test verisinde elde edilen R2 puanı
    mse_training_score: Training verisinde elde edilen MSE puanı
    mse_test_score: Test verisinde elde edilen MSE puanı
    """
    training_pred = model.predict(x_train)
    test_pred = model.predict(x_test)
    
    r2_training_score = mt.r2_score(y_train,training_pred)
    r2_test_score = mt.r2_score(y_test,test_pred)

    mse_training_score = mt.mean_squared_error(y_train,training_pred)
    mse_test_score = mt.mean_squared_error(y_test,test_pred)

    return [r2_training_score, r2_test_score, mse_training_score, mse_test_score]

In [14]:
results = score(model=lr,x_train=X_train,x_test=X_test,y_train=y_train,y_test=y_test)
print("Training R2: {:.4f} Training MSE: {:.4f}".format(results[0],results[2]))
print("Test R2: {:.4f} Test MSE: {:.4f}".format(results[1],results[3]))

Training R2: 0.8930 Training MSE: 2.9529
Test R2: 0.9138 Test MSE: 2.1538


İşte bu nokta da model tuning devreye giriyor. Burada ayarlayabileceğimiz hiperparametreler nedir öncelikle ona bakalım mesela veri kümemizi bölerken kullandığımız random_state parametresi bir hiperparametredir. Çünkü eğitim sırasında değiştirilemez ve manuel olarak uygulanan bir şeydir. Bunun gibi yapılardan başka test_size dediğimiz parametre yine veri kümesini bölerken yaptığımız belki %90-510 şeklinde bölseydik daha iyi sonuç alacaktık. Burada şuan da benim görebildiğim 2 tane var bunun dışında farklı bir metod ile veri kümesini bölebilirdik daha önce gördüğümüz K-Fold Cross-Validation şimdi onu inceleyelim.

In [28]:
lr_cv = LinearRegression()
k = 5 
# k default bir değer olarak da 5 alınır ben adımları görmek öğrenmek ve tekrar etmek için yazıyorum
iter = 1 # iterasyonları takip etmek için kullanacağım
cv = KFold(n_splits=k) 
# 3 parametre alır n_split bölme sayısı, shuffle veriyi karıştırma bool alır, random state none defa.

for training_index, test_index in cv.split(X):
    # burada split Kfold un bir fonksiyonudur bize training ve test için ayrılan veri kümelerinin 
    # indekslerini döner 

    X_train, X_test = X.iloc[training_index],X.iloc[test_index]
    y_train, y_test = y.iloc[training_index],y.iloc[test_index]
   # X_train, X_test, Y_train, Y_test = X.iloc[training_index,:], X.iloc[test_index,:], y.iloc[training_index], y.iloc[test_index]
    lr_cv.fit(X_train,y_train)

    results2 = score(model=lr_cv,x_train=X_train,x_test=X_test,y_train=y_train,y_test=y_test)

    print(f"İterasyon: {iter}")
    print("Training R2: {:.4f} Training MSE: {:.4f}".format(results2[0],results2[2]))
    print("Test R2: {:.4f} Test MSE: {:.4f}".format(results2[1],results2[3]))
    iter +=1
    # çıktı da da görüldüğü gibi 3.iterasyon en mantıklı iterasyon olarak görülmekte 
    # hem r2 hem de mse test skor u diğerlerine göre iyi. 

İterasyon: 1
Training R2: 0.9010 Training MSE: 2.7116
Test R2: 0.8787 Test MSE: 3.1365
İterasyon: 2
Training R2: 0.8904 Training MSE: 2.8897
Test R2: 0.9176 Test MSE: 2.4257
İterasyon: 3
Training R2: 0.8897 Training MSE: 3.1044
Test R2: 0.9293 Test MSE: 1.5852
İterasyon: 4
Training R2: 0.9146 Training MSE: 2.2416
Test R2: 0.8144 Test MSE: 5.4262
İterasyon: 5
Training R2: 0.8962 Training MSE: 2.8218
Test R2: 0.8955 Test MSE: 2.7911
